<h3 >Импорты</h3>

("from sklearn.experimental import enable_iterative_imputer" используется для доступа к IterativeImputer)

In [1]:
import warnings
import pickle

import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

Отключениие warnings

In [2]:
warnings.filterwarnings('ignore')

<h3 >Чтение датасета</h3>

Без колонок отброшенных на предыдущем этапе.

In [3]:
df=pd.read_csv('../data/raw/raw_data.csv').drop(columns=['duration', 'contact', 'month', 'day_of_week'])

Разобью и сохраню сырой датасет дез обработки пропусков.

In [4]:
df_train, df_test = train_test_split(df,test_size=0.3,shuffle=False)
df_train.index=range(len(df_train))
df_test.index=range(len(df_test))

df_train.to_csv('../data/train_test/train.csv',index=False)
df_test.to_csv('../data/train_test/test.csv',index=False)

Определю поднаборы признаков:

In [5]:
numeric_subset=list(df.select_dtypes(['float64','int64']).columns)
categoric_subset=list(df.select_dtypes('object').columns)

Заменю pdays=999 на -1.

In [6]:
df['pdays']=df['pdays'].replace([999],-1)

Сделаю датасает с заполненными unknown методом KNN. Признаки со значениями unknown, они есть только в категориальных признаках:

In [7]:
unknown_colomns=[]
for col in categoric_subset:
    if 'unknown' in pd.unique(df[col]):
        unknown_colomns.append(col)
unknown_colomns

['job', 'marital', 'education', 'default', 'housing', 'loan']

Заменю unknown на None. 

In [8]:
for col in unknown_colomns:
    df[col]=df[col].replace(['unknown'],None)

Проверю корректность выполнения замены:

In [9]:
df[unknown_colomns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   job        40858 non-null  object
 1   marital    41108 non-null  object
 2   education  39457 non-null  object
 3   default    32591 non-null  object
 4   housing    40198 non-null  object
 5   loan       40198 non-null  object
dtypes: object(6)
memory usage: 1.9+ MB


Разобью датасет на тренировочные и тестовые данные без перемешивания, так данные по сути основаны на времени.

In [27]:
df_train, df_test = train_test_split(df,test_size=0.3,shuffle=False)
df_train.index=range(len(df_train))
df_test.index=range(len(df_test))

Для тестовой выборки, закодирую категориальные признаки и отмасштабирую численные.

In [28]:
original_encoder=OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)
encoded_df_train_cat=pd.DataFrame(original_encoder.fit_transform(df_train[categoric_subset]),columns=categoric_subset)

num_scaler=StandardScaler()
scaled_df_train_num=pd.DataFrame(num_scaler.fit_transform(df_train[numeric_subset]))

with open('../app/encoder.pkl','wb') as f:
    pickle.dump(original_encoder,f)

with open('../app/scaler.pkl','wb') as f:
    pickle.dump(num_scaler,f)

Заменю закодированные None обратно на None, для последующего заполнения. OriginalEncoder кодирут значение None максимальным для каждого признака.

In [12]:
for col in unknown_colomns:
    encoded_df_train_cat[col]=encoded_df_train_cat[col].replace([max(encoded_df_train_cat[col])],None)

Объеденю категориальные закодированные данные с отмасштабированными числовыми для заполнения пропусков KNN.

In [13]:
df_train_encode=pd.concat([scaled_df_train_num,encoded_df_train_cat],axis=1,ignore_index=True)
df_train_encode=pd.DataFrame(df_train_encode)
df_train_encode.columns=numeric_subset+categoric_subset

Заполню пропуски с помощью KNN:

In [14]:
knn_imputer=KNNImputer()
df_train_encode_knn_imputed=pd.DataFrame(knn_imputer.fit_transform(df_train_encode),columns=df_train_encode.columns)

with open('../app/knn_imputer.pkl','wb') as f:
    pickle.dump(knn_imputer,f)

Снова выберу категориальный признаки и раскодирую их:

In [55]:
df_train_cat_knn_imputed=pd.DataFrame(original_encoder.inverse_transform(df_train_encode_knn_imputed[categoric_subset]))

Соберу снова все в один датасет, но с заполненными пропусками по методу KNN.

In [42]:
df_train_knn_imputed=pd.concat([df_train[numeric_subset],df_train_cat_knn_imputed],axis=1)
df_train_knn_imputed=pd.DataFrame(df_train_knn_imputed)
df_train_knn_imputed.columns=numeric_subset+categoric_subset

print('Checking inputer work:')
df_train_knn_imputed[unknown_colomns].info()

Checking inputer work:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28831 entries, 0 to 28830
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   job        28831 non-null  object
 1   marital    28831 non-null  object
 2   education  28831 non-null  object
 3   default    28831 non-null  object
 4   housing    28831 non-null  object
 5   loan       28831 non-null  object
dtypes: object(6)
memory usage: 1.3+ MB


Как видно, пропусков, которые были вставлены вместо значения unknown, больше нет. Пребразую соответственно и тестовый набор.

In [43]:
encoded_df_test_cat=pd.DataFrame(original_encoder.transform(df_test[categoric_subset]),columns=categoric_subset)
scaled_df_test_num=pd.DataFrame(num_scaler.transform(df_test[numeric_subset]))

for col in unknown_colomns:
    encoded_df_test_cat[col]=encoded_df_test_cat[col].replace([max(encoded_df_test_cat[col])],None)

df_test_encode=pd.concat([scaled_df_test_num,encoded_df_test_cat],axis=1,ignore_index=True)
df_test_encode=pd.DataFrame(df_test_encode)
df_test_encode.columns=numeric_subset+categoric_subset

df_test_encode_knn_imputed=pd.DataFrame(knn_imputer.transform(df_test_encode),columns=df_test_encode.columns)
df_test_cat_knn_imputed=pd.DataFrame(original_encoder.inverse_transform(df_test_encode_knn_imputed[categoric_subset]))

df_test_knn_imputed=pd.concat([df_test[numeric_subset],df_test_cat_knn_imputed],axis=1)
df_test_knn_imputed=pd.DataFrame(df_test_knn_imputed)
df_test_knn_imputed.columns=numeric_subset+categoric_subset

Созраню датасеты с обработанными пропусками по методу KNN.

In [44]:
df_train_knn_imputed.to_csv('../data/train_test/train_knn_imputer.csv',index=False)
df_test_knn_imputed.to_csv('../data/train_test/test_knn_imputer.csv',index=False)

Реализую аналогично для IterativeImputer(estimator=BayesianRidge) метода обработки пропусков.

Для тренировочного набора:

In [45]:
original_encoder=OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)
num_scaler=StandardScaler()
iter_imputer=IterativeImputer(estimator=BayesianRidge())


encoded_df_train_cat=pd.DataFrame(original_encoder.fit_transform(df_train[categoric_subset]),columns=categoric_subset)
scaled_df_train_num=pd.DataFrame(num_scaler.fit_transform(df_train[numeric_subset]))

for col in unknown_colomns:
    encoded_df_train_cat[col]=encoded_df_train_cat[col].replace([max(encoded_df_train_cat[col])],None)

df_train_encode=pd.concat([scaled_df_train_num,encoded_df_train_cat],axis=1,ignore_index=True)
df_train_encode=pd.DataFrame(df_train_encode)
df_train_encode.columns=numeric_subset+categoric_subset

df_train_encode_iter_imputed=pd.DataFrame(iter_imputer.fit_transform(df_train_encode),columns=df_train_encode.columns)
df_train_cat_iter_imputed=pd.DataFrame(original_encoder.inverse_transform(df_train_encode_iter_imputed[categoric_subset]))

df_train_iter_imputed=pd.concat([df_train[numeric_subset],df_train_cat_iter_imputed],axis=1)
df_train_iter_imputed=pd.DataFrame(df_train_iter_imputed)
df_train_iter_imputed.columns=numeric_subset+categoric_subset

Для тестового набора:

In [46]:
encoded_df_test_cat=pd.DataFrame(original_encoder.transform(df_test[categoric_subset]),columns=categoric_subset)
scaled_df_test_num=pd.DataFrame(num_scaler.transform(df_test[numeric_subset]))

for col in unknown_colomns:
    encoded_df_test_cat[col]=encoded_df_test_cat[col].replace([max(encoded_df_test_cat[col])],None)

df_test_encode=pd.concat([scaled_df_test_num,encoded_df_test_cat],axis=1,ignore_index=True)
df_test_encode=pd.DataFrame(df_test_encode)
df_test_encode.columns=numeric_subset+categoric_subset

df_test_encode_iter_imputed=pd.DataFrame(iter_imputer.transform(df_test_encode),columns=df_test_encode.columns)
df_test_cat_iter_imputed=pd.DataFrame(original_encoder.inverse_transform(df_test_encode_iter_imputed[categoric_subset]))

df_test_iter_imputed=pd.concat([df_test[numeric_subset],df_test_cat_iter_imputed],axis=1)
df_test_iter_imputed=pd.DataFrame(df_test_iter_imputed)
df_test_iter_imputed.columns=numeric_subset+categoric_subset

Проверка на корректность обработки:

In [47]:
df_train_iter_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28831 entries, 0 to 28830
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             28831 non-null  int64  
 1   campaign        28831 non-null  int64  
 2   pdays           28831 non-null  int64  
 3   previous        28831 non-null  int64  
 4   emp.var.rate    28831 non-null  float64
 5   cons.price.idx  28831 non-null  float64
 6   cons.conf.idx   28831 non-null  float64
 7   euribor3m       28831 non-null  float64
 8   nr.employed     28831 non-null  float64
 9   job             28831 non-null  object 
 10  marital         28831 non-null  object 
 11  education       28831 non-null  object 
 12  default         28831 non-null  object 
 13  housing         28831 non-null  object 
 14  loan            28831 non-null  object 
 15  poutcome        28831 non-null  object 
 16  subscribed      28831 non-null  object 
dtypes: float64(5), int64(4), object

In [48]:
df_test_iter_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12357 entries, 0 to 12356
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             12357 non-null  int64  
 1   campaign        12357 non-null  int64  
 2   pdays           12357 non-null  int64  
 3   previous        12357 non-null  int64  
 4   emp.var.rate    12357 non-null  float64
 5   cons.price.idx  12357 non-null  float64
 6   cons.conf.idx   12357 non-null  float64
 7   euribor3m       12357 non-null  float64
 8   nr.employed     12357 non-null  float64
 9   job             12357 non-null  object 
 10  marital         12357 non-null  object 
 11  education       12341 non-null  object 
 12  default         12357 non-null  object 
 13  housing         12357 non-null  object 
 14  loan            12357 non-null  object 
 15  poutcome        12357 non-null  object 
 16  subscribed      12357 non-null  object 
dtypes: float64(5), int64(4), object

In [49]:
df_train_iter_imputed.head()

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,job,marital,education,default,housing,loan,poutcome,subscribed
0,56,1,-1,0,1.1,93.994,-36.4,4.857,5191.0,housemaid,married,basic.4y,no,no,no,nonexistent,no
1,57,1,-1,0,1.1,93.994,-36.4,4.857,5191.0,services,married,high.school,no,no,no,nonexistent,no
2,37,1,-1,0,1.1,93.994,-36.4,4.857,5191.0,services,married,high.school,no,yes,no,nonexistent,no
3,40,1,-1,0,1.1,93.994,-36.4,4.857,5191.0,admin.,married,basic.6y,no,no,no,nonexistent,no
4,56,1,-1,0,1.1,93.994,-36.4,4.857,5191.0,services,married,high.school,no,no,yes,nonexistent,no


In [50]:
df_test_iter_imputed.head()

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,job,marital,education,default,housing,loan,poutcome,subscribed
0,56,2,-1,0,-1.8,93.075,-47.1,1.41,5099.1,admin.,married,basic.9y,no,yes,no,nonexistent,yes
1,47,1,-1,0,-1.8,93.075,-47.1,1.41,5099.1,blue-collar,married,basic.9y,no,yes,no,nonexistent,no
2,44,2,-1,1,-1.8,93.075,-47.1,1.41,5099.1,admin.,single,high.school,no,yes,no,failure,no
3,55,3,-1,0,-1.8,93.075,-47.1,1.41,5099.1,admin.,married,university.degree,no,yes,no,nonexistent,no
4,36,2,-1,0,-1.8,93.075,-47.1,1.41,5099.1,admin.,single,university.degree,no,yes,no,nonexistent,no


Всё хорошо. Сохранение тренировочных и тестовых данных с обработанными пропусками по методу IterativeImputer(estimator=BayesianRidge):

In [52]:
df_train_iter_imputed.to_csv('../data/train_test/train_iter_imputer.csv',index=False)
df_test_iter_imputed.to_csv('../data/train_test/test_iter_imputer.csv',index=False)

<h3 >Выводы</h3>

Были составлены три датасета - без обработки пропусков, с KNN обработкой и с Iterative обработкой пропусков, далее они были разбиты на тренеровочные и тестовые датасеты без перемешивания, так как последовательность данных обусловлена временем, после чего, полученные датасеты были сохранены в в data/train_test.